In [ ]:
from Libs.Profile.stocks import Stock, project_stock_price
from Libs.Profile.portfolio import FinancialPortfolio
import os
from Libs.global_vars import STOCKS_HISTORY_PATH

portfolio = FinancialPortfolio('verybigpack')

# Load every file
for file_stock in [f for f in os.listdir(STOCKS_HISTORY_PATH) if f.endswith('.csv')]:
    portfolio.add_stock(Stock(stock_to_load=file_stock))
print(portfolio)
portfolio.get_ammounts_with_risks()


test = portfolio.get_stock('S&P500')
test.update_stock(stock_prior_risk=0.02)
_,a = project_stock_price(stock=test, expected_year_gain=1, return_datetime=True, noise=True,
                          start_date='2025-01-01', end_date='2026-01-01', dividends_months=None)
r,b = project_stock_price(stock=test, expected_year_gain=0.5, return_datetime=True, noise=True,
                          start_date='2025-01-01', end_date='2026-01-01', dividends_months=2)

import matplotlib.pyplot as plt
plt.plot(r, a, label='linear', marker='o')
plt.plot(r,b, label='compound', marker='o')
plt.grid(alpha=0.4)
plt.legend()
plt.show()

## **Update Movements**

In [ ]:
import polars as pl
import numpy as np
from Libs.Management.bank_api import get_history_movements, update_movements_file
update_movements_file(new_orig_mov_file='data/raw_data/operazioni.xlsx', header_start=17)
get_history_movements().head()

## **Update Balance**

In [ ]:
import polars as pl
import numpy as np
from Libs.Management.bank_api import get_current_balance, get_history_balance, reconstruct_balance, update_balance_from_movements
# reconstruct_balance(current_balance=to_give) # Only for creating the balance history
update_balance_from_movements()
get_history_balance().head()

In [ ]:
from Libs.Models.data_visualization import plot_history_movements, plot_history_balance, overlay_generic_graph
from Libs.Management.bank_api import get_history_balance
from Libs.Models.data_manipulation import get_regularized_dataset

import polars as pl
import numpy as np

balance_df = get_history_balance()

def get_moving_average(dataset : pl.DataFrame, window_size=2, regularize=False) -> pl.DataFrame:
    """
    """
    dataset = dataset.with_columns(
        moving_average=pl.col(dataset.columns[-1]).rolling_mean(center=True, window_size=window_size),
    )
    return dataset

df = get_regularized_dataset(balance_df, reg_period='1d')
df_ma = get_moving_average(df, window_size=15)
plot_history_balance(balance_df=df, additional_plots=[{'plot_function' : overlay_generic_graph, 'plot_parameters':{'dataset':df_ma, 'col_to_plot':'moving_average',
                                                                                                            'color':'darkred', 'linestyle':'--','label':'Moving Average'}}])


In [ ]:
# from Libs.Models.Predictions.algorithms import linear_interpolation
from sklearn.linear_model import LinearRegression
from Libs.Models.data_manipulation import simple_linear_imputation, convert_date_to_numeric, convert_numeric_to_date

# For imputation we need numerical values
df_ma_imputed = convert_date_to_numeric(df_ma, ['date'])
df_ma_imputed = simple_linear_imputation(df_ma_imputed, 'date', 'moving_average')
df_ma_imputed = convert_numeric_to_date(df_ma_imputed, ['date'])
df_ma_imputed.head(1)

In [ ]:
def get_prediction(X : np.ndarray, model : dict, prediction_column=None):
    """
    X : X matrix to predict (if single, it should be a vector column, Ax=pred)
    """

    if prediction_column is None:
        prediction_column='model_prediction'

    prediction = model['model'].predict(X)
    A = pl.DataFrame()

    for ind, column_vector in enumerate(X.T):
        print(column_vector)
        A = A.with_columns(pl.Series(model['x_cols'][ind], column_vector))

    A = A.with_columns(pl.Series(prediction_column, prediction.T[0]))
    return A


def goal_reach_prediction(model : dict, start_date : str, max_iter=1000, goal=30_000):
    """
    # 1d = 86400000000
    """
    start_date = convert_date_to_numeric(start_date)
    initial_val = model['model'].predict(start_date)

    if goal<=initial_val:
        return start_date
    
    x = start_date
    for i in range(max_iter):
        y_pred = model['model'].predict(x.reshape(1, -1))[0]
        error = y_pred - goal
        
        if abs(error) < 10e-6:
            print(f"Converged in {i} steps")
            return x
        
        # Update x by moving against error direction (approximate)
        # We'll perturb each feature slightly to find which direction improves the output
        grad_approx = np.zeros_like(x)
        delta = 1e-4
        
        for j in range(len(x)):
            x_up = x.copy()
            x_up[j] += delta
            y_up = model.predict(x_up.reshape(1, -1))[0]
            
            # Approximate derivative
            grad_approx[j] = (y_up - y_pred) / delta
        
        # Gradient descent step (move in the direction that reduces the error)
        x -= 0.01 * error * grad_approx  # error term scales the gradient

    raise RuntimeError("Did not converge")


from Libs.Models.Predictions.algorithms import linear_fit
df_ma, linear_model = linear_fit(dataset=df_ma, cols_to_fit=None)
df_ma = convert_numeric_to_date(df_ma, ['date'])

goal_reach_prediction(linear_model, start_date=['2025-01-01'])

# X = convert_date_to_numeric(['2025-06-01', '2025-07-01', '2025-08-01'])
# pred_df = get_prediction(X, linear_model)
# pred_df = convert_numeric_to_date(pred_df, date_cols=['date'])


In [ ]:

plot_history_balance(balance_df=df_ma, balance_col='total_value', additional_plots=[
    {'plot_function':overlay_generic_graph, 'plot_parameters':{'dataset':df_ma_imputed, 'col_to_plot':'moving_average', 'label':'moving average (linear imputation)'}},
    {'plot_function':overlay_generic_graph, 'plot_parameters':{'dataset':pred_df, 'col_to_plot':'model_prediction', 'label':'linear prediction', 'marker':'o', 'linestyle':'--'}}
    ]
)

In [ ]:
df_ma.select('date').cast(pl.Datetime)